In [7]:
#!pip install tensorflow-gpu==2.0.0-alpha0
#!pip install tensorflow==2.0.0-alpha0
!pip install arabic-reshaper

  Created wheel for arabic-reshaper: filename=arabic_reshaper-2.1.1-py3-none-any.whl size=16507 sha256=85139c00f2c592f42d62a16df844dc0acf58437a6ec4a6909bf207c80cec3044
  Stored in directory: c:\users\meher\appdata\local\pip\cache\wheels\a9\c5\b5\f439511ed3062e2889e71ac6d77827269a6ba7d5dfc161a554
Successfully built arabic-reshaper


In [8]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.10 and enable eager execution
import tensorflow as tf

#tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from bidi.algorithm import get_display
import arabic_reshaper
import unicodedata
import re
import numpy as np
import os
import time
import math 

print(tf.__version__)

1.14.0


In [9]:
# Path for the dataset file
path_to_file = "araa.txt"

In [10]:
def unicode_to_ascii(s):
    '''
    Args:
        s : UniCode file
    Returns:
        ASCII file
    
    Converts the unicode file to ascii
    '''
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
    '''
    Args:
        w : A single word
    Returns:
        w : Single normalize word 
    
    Convert Unicode to ASCII
    Creating a space between a word and the punctuation following it
    eg: "he is a boy." => "he is a boy ." 
    Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    
    Replacing everything with space except (a-z, A-Z, ا-ي ".", "?", "!", ",")
    
    Adding a start and an end token to the sentence
    
    '''
    w = unicode_to_ascii(w.lower().strip())
    
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    #w = re.sub(r"[^a-zA-Z؀-ۿ?.!,¿]+", " ", w)
    w = w.rstrip().strip()
    
    w = '<start> %s <end>'%w
    return w

In [19]:
def create_dataset(path, num_examples):
    '''
    Args:
        path: Path of the dataset file
        num_examples: Threshold to read a range from dataset
    Returns:
        word_pairs : All readed words from dataset as a pairs 
        
    1. Remove the accents
    2. Clean the sentences
    3. Return word pairs in the format: [ENGLISH, ARABIC]
    '''
    lines = open(path, encoding='windows-1256').read().strip().split('\n')
    word_pairs = [[preprocess_sentence(w) for w in l.split('#')]  for l in lines[:num_examples]]
    print(len(lines))
    print(np.array(word_pairs).shape)
    return zip(*word_pairs)

In [16]:
class LanguageIndex():
    ''' Creates a word -> index mapping

    This class creates a word -> index mapping 
    (e.g., "dad" -> 5) and vice-versa 
    (e.g., 5 -> "dad") for each language,
    
    Attributes:
        lang: A langauage to map to its index.
    '''
    def __init__(self, lang):
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()

        self.create_index()
    
    def create_index(self):
        for phrase in self.lang:
            self.vocab.update(phrase.split(' '))
    
        self.vocab = sorted(self.vocab)
    
        self.word2idx['<pad>'] = 0
        
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1
    
        for word, index in self.word2idx.items():
            self.idx2word[index] = word

In [17]:
def max_length(tensor):
    return max(len(t) for t in tensor)
def load_dataset(path, num_examples):
    # creating cleaned input, output pairs
    pairs = create_dataset(path, num_examples)
    # index language using the class defined above    
    np_lang = create_dataset(path, num_examples)
    targ_lang = create_dataset(path, num_examples)
    # Vectorize the input and target languages
    # Spanish sentences
    input_tensor = [[inp_lang.word2idx[s] for s in sp.split(' ')] for en, sp in pairs]
    # English sentences
    target_tensor = [[targ_lang.word2idx[s] for s in en.split(' ')] for en, sp in pairs]
    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor,maxlen=max_length_inp,padding='post')
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor,maxlen=max_length_tar,padding='post')
    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar


In [20]:
# Try experimenting with the size of that dataset
num_examples = 1755
# Load the dataset and return Tensor of the input, Tensor for the traget, Indexed input, Indexed traget, Max length input, Max length target
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(path_to_file, num_examples)

1755
(1755,)
1755
(1755,)
1755
(1755,)


ValueError: too many values to unpack (expected 2)

In [ ]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 24
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 300
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
def gru(units):
    
    '''
    Args: 
        units: number of units
    Return:
        GRU units
    
    If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
    the code automatically does that.
    '''
    return tf.keras.layers.GRU(units, 
                                return_sequences=True, 
                                return_state=True, 
                                recurrent_activation='sigmoid', 
                                recurrent_initializer='glorot_uniform')

In [ ]:
class Encoder(tf.keras.Model):
    ''' Create the word embeddings, then fed as input into the main network,
    
    This class create encoder which consists of two multi-layer RNNs – an encoder 
    for the source language and a decoder for the target language.
    
    Attributes:
        vocab_size: Size of vocabulary
        embedding_dim: Embedding vector length
        enc_units:  Number of GRUs units
        batch_sz: batch size ** minimize it for low RAM
    '''
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
class Decoder(tf.keras.Model):
    ''' Create the decoder, also needs to have access to the source information.
    
    This class create decoder which consists of decoder_cell (similar to encoder_cell), 
    a helper, and the previous encoder_state as inputs.
    
    Attributes:
        vocab_size: Size of vocabulary
        embedding_dim: Embedding vector length
        enc_units:  Number of GRUs units
        batch_sz: batch size ** minimize it for low RAM
    '''
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
            
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))
    
    def saved_hidden_state(self):
        return self.gru.states

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

In [ ]:
optimizer = tf.keras.optimizers.Adam()


def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [ ]:
save_dir = './model_save'
encoder_prefix = os.path.join(save_dir, "encoder_weights")
decoder_prefix = os.path.join(save_dir, "decoder_weights")

In [ ]:
encoder.load_weights(encoder_prefix)
decoder.load_weights(decoder_prefix)

In [ ]:
EPOCHS = 8
best_total_loss = 0.0071

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1, total_loss / N_BATCH))
    
    if best_total_loss > (total_loss / N_BATCH):
        
        best_total_loss = total_loss
        
        encoder.save_weights(encoder_prefix)
        decoder.save_weights(decoder_prefix)
        
        print('Saving weights at epoch {} with Loss {:.4f}'.format(epoch + 1, total_loss / N_BATCH))
        
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

In [ ]:
def evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    
    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word2idx[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weigths to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [ ]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    
    heatmap = ax.matshow(attention, cmap='rainbow')

    for y in range(attention.shape[0]):
        for x in range(attention.shape[1]):
            ax.text(x , y, '%.4f' % attention[y, x],
                     horizontalalignment='center',
                     verticalalignment='center', color='black')

    fig.colorbar(heatmap)

    fontdict = {'fontsize': 14}
    
    ax.set_xticklabels([''] + sentence, fontdict=fontdict)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    plt.show()

In [ ]:
def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    result, sentence, attention_plot = evaluate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
        
    print('Input: {}'.format(sentence))
    print('Predicted translation: {}'.format(result))

    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, get_display(arabic_reshaper.reshape(sentence)).split(' '), result.split(' '))

In [ ]:
translate(u'جرب', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)

In [ ]:
translate(u'اهلا وسهلا', encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)